In [2]:
import json
import numpy as np
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import pickle
import os
import math
import random
from eoh.problems.optimization.classic_benchmark_path_planning.utils.benchmark import MultiMapBenchmarker
import warnings
import types
import sys

In [3]:
class Map(NamedTuple):
    grid: np.ndarray
    start: Union[Tuple[float, float], Tuple[float, float, float]]
    goal: Union[Tuple[float, float], Tuple[float, float, float]]
    obstacles: List[Union[Tuple[float, float, float, float], Tuple[float, float, float, float, float, float]]] # x, y, width, height or x, y, z, width, height, dimension
    size: Union[Tuple[int, int], Tuple[int, int, int]]

class PlannerResult(NamedTuple):
    success: bool
    path: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    nodes: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    edges: List[Tuple[Tuple[float, ...], Tuple[float, ...]]]  # (parent, child)

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True  # 장애물 충돌 여부 등


In [14]:
def visualize_map_shapes(
    map_array: np.ndarray,
    start: Optional[Tuple[int, ...]] = None,
    goal: Optional[Tuple[int, ...]] = None,
    obs:List[Tuple[int, ...]] = None,
    path: Optional[List[Tuple[float, ...]]] = None,
    nodes: Optional[List[Tuple[float, ...]]] = None,
    edges: Optional[List[Tuple[float, ...]]] = None,
    title: str = "Map Visualization"
):
    fig = go.Figure()

    if map_array.ndim == 2:
        height, width = map_array.shape
        
        for x, y, w, h in obs:
                fig.add_shape(
                    type="rect",
                    x0=x, x1=x+w, y0=y, y1=y+h,
                    fillcolor="purple",opacity=0.5,
                    line=dict(width=0)
                )

        # 방문 노드
        if nodes:
            vx, vy = zip(*nodes)
            fig.add_trace(go.Scatter(
                x=vx, y=vy, mode="markers",
                marker=dict(size=4, color="blue"),
                name="nodes"
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter(
                    x=[parent.position[0], child.position[0]], y=[parent.position[1], child.position[1]],
                    mode="lines",
                    line=dict(color="lightblue", width=1),
                    showlegend=False,
                    hoverinfo="skip"
                ))
                
        # 경로
        if path:
            px, py = zip(*path)
            fig.add_trace(go.Scatter(
                x=px, y=py, mode="lines+markers",
                line=dict(color="green"),
                marker=dict(size=6),
                name="Path"
            ))


        # 시작/목표
        if start:
            fig.add_trace(go.Scatter(
                x=[start[0]], y=[start[1]], mode="markers",
                marker=dict(size=10, color="red"),
                name="Start"
            ))

        if goal:
            fig.add_trace(go.Scatter(
                x=[goal[0]], y=[goal[1]], mode="markers",
                marker=dict(size=10, color="orange"),
                name="Goal"
            ))

        fig.add_shape(
            type="rect",
            x0=0, y0=0,
            x1=width, y1=height,
            line=dict(color="white", width=3),
            fillcolor="rgba(0,0,0,0)",  # 투명 내부
            layer="above"
        )

        fig.update_layout(
            title=title,
            xaxis=dict(scaleanchor="y", showgrid=False),
            # yaxis=dict(showgrid=False, autorange="reversed"),
            yaxis=dict(showgrid=False),
            height=600, width=600
        )

    elif map_array.ndim == 3:
        z, y, x = map_array.nonzero()
        x, y, z = list(x), list(y), list(z)

        # for x,y,w,h in obs:
        #     fig.add_trace(go.Mesh3d(
        #         x=x, y=y, z=z,
        #         color='black',
        #         opacity=1.0,
        #         alphahull=0,
        #         name='Obstacles'
        #     ))

        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='black',
            opacity=1.0,
            alphahull=0,
            name='Obstacles'
        ))

        if nodes:
            vx, vy, vz = zip(*nodes)
            fig.add_trace(go.Scatter3d(
                x=vx, y=vy, z=vz,
                mode='markers',
                marker=dict(size=2, color='blue'),
                name='Visited'
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter3d(
                    x=[parent.position[0], child.position[0]],
                    y=[parent[1].position, child.position[1]],
                    z=[parent[2].position, child.position[2]],
                    mode='lines',
                    line=dict(color='lightblue', width=2),
                    showlegend=False,
                    hoverinfo="skip"
                ))

        if path:
            px_, py_, pz_ = zip(*path)
            fig.add_trace(go.Scatter3d(
                x=px_, y=py_, z=pz_,
                mode='lines+markers',
                marker=dict(size=3, color='green'),
                name='Path'
            ))

        if start:
            fig.add_trace(go.Scatter3d(
                x=[start[0]], y=[start[1]], z=[start[2]],
                mode='markers',
                marker=dict(size=5, color='red'),
                name='Start'
            ))

        if goal:
            fig.add_trace(go.Scatter3d(
                x=[goal[0]], y=[goal[1]], z=[goal[2]],
                mode='markers',
                marker=dict(size=5, color='orange'),
                name='Goal'
            ))

        fig.update_layout(
            title=title,
            scene=dict(aspectmode='data'),
            height=700, width=700
        )

        

    fig.show()


In [5]:
class MapIO:
    @staticmethod
    def save_map(map_data: Map, filename: str) -> None:
        """Save Map object to a binary file."""
        with open(filename, 'wb') as f:
            pickle.dump(map_data, f)

    @staticmethod
    def load_map(filename: str) -> Map:
        """Load Map object from a binary file."""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Map file not found: {filename}")
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [6]:
raw_maps = ["Maze_map_easy.pkl", "Narrow_map.pkl", "Multi_obs_map.pkl"]

multi_obs_map = MapIO.load_map("Multi_obs_map.pkl")
print("Start:", multi_obs_map.start)
print("Obstacles:", len(multi_obs_map.obstacles))
print(multi_obs_map.grid.shape)

hard_maze_map = MapIO.load_map("Maze_map.pkl")
print("Start:", hard_maze_map.start)
print("Obstacles:", len(hard_maze_map.obstacles))
print(hard_maze_map.grid.shape)


maze_map = MapIO.load_map("Maze_map_easy.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

narrow_map = MapIO.load_map("Narrow_map.pkl")
print("Start:", narrow_map.start)
print("Obstacles:", len(narrow_map.obstacles))
print(narrow_map.grid.shape)

Start: (1, 1)
Obstacles: 35
(100, 100)
Start: (29, 40)
Obstacles: 8
(100, 100)
Start: (29, 10)
Obstacles: 5
(100, 100)
Start: (80, 50)
Obstacles: 5
(100, 100)


In [7]:
maps=[multi_obs_map, maze_map, narrow_map]

In [6]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method__.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [14]:
print([n['algorithm'] for n in classic_method])

['RRT', 'RRT*', 'RRT-Connect', 'RRT*-Connect', 'BI-RRT', 'BI-RRT*', 'Informed-RRT*', 'Informed-RRT*-Connect', 'Bidirectional-Informed-RRT*', 'Improved-RRT*-Connect']


In [14]:
for method in classic_method:
    if method['algorithm'] == 'Informed-RRT*':
     code = method['code']
     break

In [8]:
import_string ='''
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set, TYPE_CHECKING, Type
import time
from queue import Queue
import numpy as np
import random
import math
import sys
import os
from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult, Map
'''

In [ ]:
# code = classic_method[-1]['code']

In [19]:
code = "class Node:\n    def __init__(self, position, parent=None, cost=0.0):\n        self.position = tuple(position)\n        self.parent = parent\n        self.cost = float(cost)\n        self.children = []\n        if parent is not None:\n            parent.children.append(self)\n\n    def attach(self, new_parent):\n        if new_parent is self.parent:\n            return\n        if self.parent is not None:\n            try:\n                self.parent.children.remove(self)\n            except:\n                pass\n        self.parent = new_parent\n        if new_parent is not None:\n            if self not in new_parent.children:\n                new_parent.children.append(self)\n\n    def is_ancestor_of(self, other):\n        cur = other\n        while cur is not None:\n            if cur is self:\n                return True\n            cur = cur.parent\n        return False\n\n    def propagate_cost(self, delta):\n        if delta == 0.0:\n            return\n        self.cost += delta\n        for c in self.children:\n            c.propagate_cost(delta)\nclass Planner:\n    def __init__(\n        self,\n        max_iter=3000,\n        step_size=10.0,\n        goal_bias=0.1,\n        informed_bias=0.5,\n        dupe_radius_ratio=0.5,\n        neighbor_radius_factor=2.5,\n        k_near=5,\n        shortcut_attempts=80\n    ):\n        self.max_iter = int(max_iter)\n        self.step_size = float(step_size)\n        self.goal_bias = float(goal_bias)\n        self.informed_bias = float(informed_bias)\n        self.dupe_radius_ratio = float(dupe_radius_ratio)\n        self.neighbor_radius_factor = float(neighbor_radius_factor)\n        self.k_near = int(k_near)\n        self.shortcut_attempts = int(shortcut_attempts)\n        self._rng = 123456789\n\n    def plan(self, map):\n        self.bounds = tuple(map.size)\n        self.dims = len(self.bounds)\n        self.is3d = (self.dims == 3)\n        self.start = tuple(map.start)\n        self.goal = tuple(map.goal)\n        self.obstacles = list(map.obstacles) if map.obstacles else []\n\n        if not self._in_bounds(self.start) or not self._in_bounds(self.goal):\n            return PlannerResult(False, [], [], [])\n        if self._point_in_obstacle(self.start) or self._point_in_obstacle(self.goal):\n            return PlannerResult(False, [], [], [])\n\n        self._seed_rng()\n        self.dupe_radius = max(0.5, self.step_size * self.dupe_radius_ratio)\n        self.neighbor_radius = max(self.step_size * 1.5, self.step_size * self.neighbor_radius_factor)\n\n        if self._edge_free(self.start, self.goal):\n            n0 = Node(self.start, None, 0.0)\n            n1 = Node(self.goal, n0, self._dist(self.start, self.goal))\n            edges = [(n0, n1)]\n            return PlannerResult(True, self._finalize_path([self.start, self.goal]), [n0, n1], edges)\n\n        nodes = []\n        edges = []\n\n        a_root = Node(self.start, None, 0.0)\n        b_root = Node(self.goal, None, 0.0)\n        nodes.extend([a_root, b_root])\n\n        A = [a_root]\n        B = [b_root]\n\n        best_len = float('inf')\n\n        for it in range(self.max_iter):\n            grow_A = (it % 2 == 0)\n            T = A if grow_A else B\n            O = B if grow_A else A\n            attractor = self.goal if grow_A else self.start\n\n            qrand = self._sample(attractor, best_len)\n            if qrand is None:\n                continue\n\n            qnear = self._nearest_in(T, qrand)\n            if qnear is None:\n                continue\n\n            qnew_pos = self._steer(qnear.position, qrand)\n            if not self._in_bounds(qnew_pos):\n                continue\n            if self._point_in_obstacle(qnew_pos):\n                continue\n            if self._has_near_in(T, qnew_pos, self.dupe_radius):\n                continue\n\n            parent = self._best_parent(T, qnew_pos, qnear)\n            if parent is None:\n                continue\n            if not self._edge_free(parent.position, qnew_pos):\n                continue\n\n            qnew_cost = parent.cost + self._dist(parent.position, qnew_pos)\n            qnew = Node(qnew_pos, parent, qnew_cost)\n            T.append(qnew)\n            nodes.append(qnew)\n            edges.append((parent, qnew))\n\n            self._light_rewire(T, qnew, edges)\n\n            ok, last_other, added = self._connect_toward(O, qnew.position, nodes, edges)\n            if ok and last_other is not None:\n                edges.append((last_other, qnew))\n                path = self._extract_path(qnew, last_other)\n                best_len = self._path_len(path)\n                return PlannerResult(True, self._finalize_path(path), nodes, edges)\n\n        return PlannerResult(False, [], nodes, edges)\n\n    # RNG\n    def _seed_rng(self):\n        s = 0xA5A5A5A5\n        for v in self.start + self.goal:\n            s ^= (int(v * 1315423911) + (s << 7) + (s >> 3)) & 0xffffffff\n            s &= 0xffffffff\n        s ^= (len(self.obstacles) * 0x9E3779B9) & 0xffffffff\n        if s == 0:\n            s = 123456789\n        self._rng = s & 0xffffffff\n\n    def _rand(self):\n        self._rng = (1664525 * self._rng + 1013904223) & 0xffffffff\n        return self._rng / float(1 << 32)\n\n    def _rand_range(self, a, b):\n        return a + (b - a) * self._rand()\n\n    # Geometry\n    def _dist(self, a, b):\n        s = 0.0\n        for i in range(self.dims):\n            d = a[i] - b[i]\n            s += d * d\n        return s ** 0.5\n\n    def _in_bounds(self, p):\n        for i in range(self.dims):\n            if p[i] < 0.0 or p[i] > self.bounds[i]:\n                return False\n        return True\n\n    def _steer(self, a, b):\n        d = self._dist(a, b)\n        if d <= 1e-12:\n            tgt = a\n        elif d <= self.step_size:\n            tgt = b\n        else:\n            r = self.step_size / d\n            tgt = tuple(a[i] + (b[i] - a[i]) * r for i in range(self.dims))\n        out = tuple(max(0.0, min(tgt[i], self.bounds[i])) for i in range(self.dims))\n        return out\n\n    # Nearest and neighbors\n    def _nearest_in(self, nodes_list, pos):\n        best = None\n        bd = 1e100\n        for n in nodes_list:\n            d = self._dist(n.position, pos)\n            if d < bd:\n                bd = d\n                best = n\n        return best\n\n    def _k_nearest_in(self, nodes_list, pos, k):\n        if not nodes_list:\n            return []\n        pairs = []\n        for n in nodes_list:\n            pairs.append((self._dist(n.position, pos), n))\n        pairs.sort(key=lambda x: x[0])\n        out = [n for _, n in pairs[:max(1, k)]]\n        return out\n\n    def _neighbors_radius(self, nodes_list, pos, radius):\n        out = []\n        r = float(radius)\n        for n in nodes_list:\n            if self._dist(n.position, pos) <= r:\n                out.append(n)\n        return out\n\n    def _has_near_in(self, nodes_list, pos, radius):\n        r = float(radius)\n        for n in nodes_list:\n            if self._dist(n.position, pos) <= r:\n                return True\n        return False\n\n    # Parent selection\n    def _best_parent(self, nodes_list, qpos, fallback):\n        cands = self._k_nearest_in(nodes_list, qpos, self.k_near)\n        if fallback not in cands:\n            cands.append(fallback)\n        best = None\n        best_cost = 1e100\n        for p in cands:\n            if not self._edge_free(p.position, qpos):\n                continue\n            c = p.cost + self._dist(p.position, qpos)\n            if c < best_cost:\n                best_cost = c\n                best = p\n        return best\n\n    # Light rewiring\n    def _light_rewire(self, nodes_list, qnew, edges):\n        nbrs = self._neighbors_radius(nodes_list, qnew.position, self.neighbor_radius)\n        rewired = 0\n        for w in nbrs:\n            if w is qnew or w is qnew.parent:\n                continue\n            if w.is_ancestor_of(qnew):\n                continue\n            d = self._dist(qnew.position, w.position)\n            cand_cost = qnew.cost + d\n            if cand_cost + 1e-12 < w.cost and self._edge_free(qnew.position, w.position):\n                oldp = w.parent\n                self._replace_edge(edges, oldp, w, qnew)\n                w.attach(qnew)\n                delta = cand_cost - w.cost\n                w.propagate_cost(delta)\n                rewired += 1\n                if rewired >= 10:\n                    break\n\n    # Connect toward a target (other tree growth)\n    def _connect_toward(self, tree_nodes, target_pos, all_nodes, edges):\n        if not tree_nodes:\n            return False, None, []\n        cur = self._nearest_in(tree_nodes, target_pos)\n        added = []\n        for _ in range(1000):\n            nxt = self._steer(cur.position, target_pos)\n            if self._dist(cur.position, nxt) <= 1e-9:\n                break\n            if not self._in_bounds(nxt):\n                break\n            if self._point_in_obstacle(nxt):\n                break\n            if self._has_near_in(tree_nodes, nxt, self.dupe_radius):\n                # If very close to some node, snap to the nearest and continue\n                cur = self._nearest_in(tree_nodes, nxt)\n                if self._dist(cur.position, target_pos) <= self.step_size and self._edge_free(cur.position, target_pos):\n                    return True, cur, added\n                continue\n            if not self._edge_free(cur.position, nxt):\n                break\n            new_cost = cur.cost + self._dist(cur.position, nxt)\n            q = Node(nxt, cur, new_cost)\n            tree_nodes.append(q)\n            all_nodes.append(q)\n            edges.append((cur, q))\n            added.append(q)\n            cur = q\n            if self._dist(cur.position, target_pos) <= self.step_size and self._edge_free(cur.position, target_pos):\n                return True, cur, added\n        return False, cur, added\n\n    # Collision\n    def _point_in_obstacle(self, p):\n        if self.is3d:\n            px, py, pz = p\n            for x, y, z, w, h, d in self.obstacles:\n                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:\n                    return True\n            return False\n        else:\n            px, py = p\n            for x, y, w, h in self.obstacles:\n                if x <= px <= x + w and y <= py <= y + h:\n                    return True\n            return False\n\n    def _edge_free(self, a, b):\n        if self.is3d:\n            for x, y, z, w, h, d in self.obstacles:\n                if self._seg_aabb_intersect_3d(a, b, (x, y, z), (x + w, y + h, z + d)):\n                    return False\n            return True\n        else:\n            for x, y, w, h in self.obstacles:\n                if self._seg_aabb_intersect_2d(a, b, (x, y), (x + w, y + h)):\n                    return False\n            return True\n\n    def _seg_aabb_intersect_2d(self, p0, p1, bmin, bmax):\n        tmin = 0.0\n        tmax = 1.0\n        for i in range(2):\n            p = p0[i]\n            d = p1[i] - p0[i]\n            mn = bmin[i]\n            mx = bmax[i]\n            if abs(d) < 1e-12:\n                if p < mn or p > mx:\n                    return False\n            else:\n                inv = 1.0 / d\n                t1 = (mn - p) * inv\n                t2 = (mx - p) * inv\n                if t1 > t2:\n                    t1, t2 = t2, t1\n                if t1 > tmin:\n                    tmin = t1\n                if t2 < tmax:\n                    tmax = t2\n                if tmin > tmax:\n                    return False\n        return True\n\n    def _seg_aabb_intersect_3d(self, p0, p1, bmin, bmax):\n        tmin = 0.0\n        tmax = 1.0\n        for i in range(3):\n            p = p0[i]\n            d = p1[i] - p0[i]\n            mn = bmin[i]\n            mx = bmax[i]\n            if abs(d) < 1e-12:\n                if p < mn or p > mx:\n                    return False\n            else:\n                inv = 1.0 / d\n                t1 = (mn - p) * inv\n                t2 = (mx - p) * inv\n                if t1 > t2:\n                    t1, t2 = t2, t1\n                if t1 > tmin:\n                    tmin = t1\n                if t2 < tmax:\n                    tmax = t2\n                if tmin > tmax:\n                    return False\n        return True\n\n    # Edges\n    def _replace_edge(self, edges, old_parent, child, new_parent):\n        if old_parent is not None:\n            rm = -1\n            for i, (u, v) in enumerate(edges):\n                if u is old_parent and v is child:\n                    rm = i\n                    break\n            if rm >= 0:\n                edges.pop(rm)\n        if new_parent is not None:\n            exists = False\n            for u, v in edges:\n                if u is new_parent and v is child:\n                    exists = True\n                    break\n            if not exists:\n                edges.append((new_parent, child))\n\n    # Sampling\n    def _sample(self, attractor, best_len):\n        for _ in range(24):\n            r = self._rand()\n            if best_len < float('inf') and r < self.informed_bias:\n                p = self._ellipse_sample(self.start, self.goal, best_len * 1.02)\n                if p is not None and not self._point_in_obstacle(p):\n                    return p\n            else:\n                if r < self.goal_bias:\n                    p = attractor\n                else:\n                    p = tuple(self._rand_range(0.0, self.bounds[i]) for i in range(self.dims))\n                if self._in_bounds(p) and not self._point_in_obstacle(p):\n                    return p\n        return None\n\n    def _ellipse_sample(self, f1, f2, max_sum):\n        c = tuple(0.5 * (f1[i] + f2[i]) for i in range(self.dims))\n        half = 0.5 * max_sum\n        ext = [half] * self.dims\n        for _ in range(16):\n            p = tuple(c[i] + self._rand_range(-ext[i], ext[i]) for i in range(self.dims))\n            if not self._in_bounds(p):\n                continue\n            if (self._dist(p, f1) + self._dist(p, f2)) <= max_sum:\n                return p\n        return None\n\n    # Path ops\n    def _path_to_root(self, node):\n        pts = []\n        cur = node\n        while cur is not None:\n            pts.append(cur.position)\n            cur = cur.parent\n        return pts[::-1]\n\n    def _extract_path(self, a_node, b_node):\n        pa = self._path_to_root(a_node)\n        pb = self._path_to_root(b_node)\n        return pa + pb[::-1]\n\n    def _path_len(self, path):\n        L = 0.0\n        for i in range(1, len(path)):\n            L += self._dist(path[i - 1], path[i])\n        return L\n\n    def _visibility_prune(self, path):\n        if len(path) < 3:\n            return list(path)\n        pruned = [path[0]]\n        i = 0\n        n = len(path)\n        while i < n - 1:\n            j = n - 1\n            advanced = False\n            while j > i + 1:\n                if self._edge_free(pruned[-1], path[j]):\n                    pruned.append(path[j])\n                    i = j\n                    advanced = True\n                    break\n                j -= 1\n            if not advanced:\n                pruned.append(path[i + 1])\n                i += 1\n        return pruned\n\n    def _shortcut(self, path, attempts):\n        if len(path) < 3:\n            return path[:]\n        pts = path[:]\n        best_L = self._path_len(pts)\n        tries = 0\n        while tries < attempts and len(pts) > 2:\n            n = len(pts)\n            i = int(self._rand_range(0, n - 2))\n            j = int(self._rand_range(i + 2, n))\n            if j >= n:\n                j = n - 1\n            if j <= i + 1:\n                tries += 1\n                continue\n            a = pts[i]\n            b = pts[j]\n            if self._edge_free(a, b):\n                cand = pts[:i + 1] + pts[j:]\n                L = self._path_len(cand)\n                if L <= best_L + 1e-12:\n                    pts = cand\n                    best_L = L\n            tries += 1\n        return pts\n\n    def _finalize_path(self, path):\n        if not path:\n            return []\n        p = self._visibility_prune(path)\n        p = self._shortcut(p, self.shortcut_attempts)\n        return p"

In [22]:
code_string = code
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    planning_module = types.ModuleType("planning_module")
    exec(import_string+code_string, planning_module.__dict__)
    sys.modules[planning_module.__name__] = planning_module
    p = planning_module.Planner(max_iter=5000)

In [23]:
for pmap in maps:
    result = p.plan(map=pmap)
    visualize_map_shapes(pmap.grid, obs=pmap.obstacles, start=pmap.start, goal=pmap.goal,
                        path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [18]:
for pmap in maps:
    result = p.plan(map=pmap)
    visualize_map_shapes(pmap.grid, obs=pmap.obstacles, start=pmap.start, goal=pmap.goal,
                        path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [65]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method_.json'

In [34]:
result = classic_method[7]

In [105]:
result = {'algorithm': 'Improved-RRT*-Connect',
 'algorithm_description': 'This algorithm with post-optimization: a bidirectional RRT*-Connect that, after finding a first feasible “best shot” path, keeps optimizing for only a user-defined number of extra iterations (or until no further improvements occur), then terminates—while maintaining strict collision checks and a 30 s time cap.',
 'planning_mechanism': 'Mechanism: Alternate tree growth with RRT* best-parent selection and local rewiring; greedily connect the opposite tree; upon the first connection, record the best path and continue only N additional iterations (or until M consecutive non-improving attempts), always staying within bounds and validating node and edge collisions; stop early on time limit and return the best path so far.',
 'code': code}

In [35]:
with open(json_path, "a") as f:
    json.dump(result, f, indent=4)
    f.write(",\n")

In [33]:
classic_method[7]['code'] = code

In [58]:
code=''''''

In [12]:
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=100)

total_df = pd.DataFrame()

# for method in classic_method[:-2]:
for method in classic_method:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['algorithm']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        planner = planning_module.Planner(max_iter=5000)
        res, avg_rest = benchmarker.run(planner.plan)
        if method['algorithm'] == 'RRT':
            ref_avg = avg_rest
            outputs = []
        outputs = benchmarker.get_results()
        if avg_rest is None: continue
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        total_df = pd.concat([total_df, res_df], axis=0)

total_df


[2025.09.15 - 15:18:21] Map 1
Iteration 1: Time taken: 0.0259 seconds, Success: True
Iteration 2: Time taken: 0.0417 seconds, Success: True
Iteration 3: Time taken: 0.0208 seconds, Success: True
Iteration 4: Time taken: 0.0197 seconds, Success: True
Iteration 5: Time taken: 0.0110 seconds, Success: True
Iteration 6: Time taken: 0.0136 seconds, Success: True
Iteration 7: Time taken: 0.0111 seconds, Success: True
Iteration 8: Time taken: 0.0502 seconds, Success: True
Iteration 9: Time taken: 0.0171 seconds, Success: True
Iteration 10: Time taken: 0.0635 seconds, Success: True
Iteration 11: Time taken: 0.0308 seconds, Success: True
Iteration 12: Time taken: 0.0236 seconds, Success: True
Iteration 13: Time taken: 0.0220 seconds, Success: True
Iteration 14: Time taken: 0.0490 seconds, Success: True
Iteration 15: Time taken: 0.0177 seconds, Success: True
Iteration 16: Time taken: 0.0309 seconds, Success: True
Iteration 17: Time taken: 0.0023 seconds, Success: True
Iteration 18: Time taken: 0

KeyboardInterrupt: 

In [17]:
total_df1 = total_df

In [19]:
total_df1

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
0,RRT,0,1.00,0.035257,461.98,188.080816,0.006254,0.0,-0.000000,-0.000000,0.000000,0.000000
1,RRT,1,0.00,0.383815,1704.46,NaN,1.000000,0.0,-0.000000,NaN,0.000000,NaN
2,RRT,2,1.00,0.214382,1646.97,305.294357,0.003862,0.0,-0.000000,-0.000000,0.000000,0.000000
3,RRT,3,1.00,0.068837,839.78,152.790835,0.007785,0.0,-0.000000,-0.000000,0.000000,0.000000
0,RRT*,0,1.00,0.069093,432.49,157.533630,0.014613,0.0,-95.968119,16.241521,133.670537,-24.873779
1,RRT*,1,0.00,0.648108,1683.19,NaN,1.000000,0.0,-68.859471,NaN,0.000000,NaN
2,RRT*,2,1.00,0.425404,1630.38,225.392740,0.011241,0.0,-98.432276,26.171993,191.051088,-23.340029
3,RRT*,3,1.00,0.125224,800.93,116.560962,0.021343,0.0,-81.914782,23.712072,174.162207,-18.961209
0,RRT-Connect,0,1.00,0.017552,116.02,196.961209,0.011149,0.0,50.216258,-4.721584,78.282236,14.511972
1,RRT-Connect,1,1.00,0.200972,1050.80,532.772541,0.003773,100.0,47.638209,NaN,-99.622699,NaN


In [ ]:
total_df1[total_df['map_id'] == 1]

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
1,RRT,1,0.00,0.383815,1704.46,NaN,1.000000,0.0,-0.000000,NaN,0.000000,NaN
1,RRT*,1,0.00,0.648108,1683.19,NaN,1.000000,0.0,-68.859471,NaN,0.000000,NaN
1,RRT-Connect,1,1.00,0.200972,1050.80,532.772541,0.003773,100.0,47.638209,NaN,-99.622699,NaN
1,RRT*-Connect,1,1.00,0.382580,1019.90,402.654093,0.006680,100.0,0.321630,NaN,-99.331971,NaN
1,BI-RRT,1,1.00,0.122854,997.55,538.847689,0.003646,100.0,67.991237,NaN,-99.635383,NaN
1,BI-RRT*,1,1.00,0.939351,991.02,406.871172,0.006614,100.0,-144.740653,NaN,-99.338567,NaN
1,Informed-RRT*,1,0.42,3.149493,2376.59,373.522374,0.584920,42.0,-720.576273,NaN,-41.508049,NaN
1,Informed-RRT*-Connect,1,0.98,3.011959,1045.81,385.133193,0.035209,98.0,-684.742899,NaN,-96.479126,NaN
1,Bidirectional-Informed-RRT*,1,1.00,0.587318,1044.63,403.448162,0.007151,100.0,-53.021275,NaN,-99.284899,NaN
1,Improved-RRT*-Connect,1,0.92,0.628170,1069.91,396.054457,0.086717,92.0,-63.664824,NaN,-91.328337,NaN


In [20]:
# classic method result
grouped_avg = total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
BI-RRT,1.0,1.0,0.016286,155.170000,213.941368,0.010032,0.0,88.471336,-0.007719,67.326861,26.876491
BI-RRT*,1.0,1.0,0.055723,175.796667,186.840439,0.016921,0.0,49.345831,12.775225,180.716786,18.262378
Bidirectional-Informed-RRT*,1.0,1.0,0.149482,500.273333,171.859631,0.029741,0.0,-95.435586,19.815096,389.512360,-22.720095
Improved-RRT*-Connect,1.0,1.0,0.146029,518.830000,170.811093,0.030153,0.0,-62.089841,19.889434,394.092078,-12.678605
Informed-RRT*,1.0,1.0,0.544449,930.400000,161.982690,0.030676,0.0,-513.705904,23.838074,415.824461,-147.265034
Informed-RRT*-Connect,1.0,1.0,0.631165,541.140000,165.450936,0.044705,0.0,-712.473861,22.505155,634.562555,-206.068314
RRT,1.0,1.0,0.180863,986.326667,214.254234,0.005997,0.0,0.000000,0.000000,0.000000,0.000000
RRT*,1.0,1.0,0.471838,958.216667,166.655759,0.015543,0.0,-177.987660,21.518749,162.073757,-48.282179
RRT*-Connect,1.0,1.0,0.050163,227.040000,187.996802,0.016492,0.0,59.113488,11.617421,176.692995,20.940995


In [20]:
(986.32 - 518)/986.32

0.4748154757076811

In [ ]:
results = [outputs[35],outputs[195],outputs[265]]

In [ ]:
for pmap, result in zip(maps, results):
    
    visualize_map_shapes(pmap.grid, obs=pmap.obstacles, start=pmap.start, goal=pmap.goal,
                        path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)